In [0]:
from pyspark.sql.functions import avg, count, col

#Read Silver Delta Table
df_silver = spark.read.format("delta").load("/FileStore/lending/silver/delta")

In [0]:

#  Aggregation Example 1 – Loan amount & interest rate by grade
loan_summary_by_grade = df_silver.groupBy("grade").agg(
    avg("loan_amnt").alias("avg_loan_amount"),
    avg("int_rate").alias("avg_interest_rate")
)

# Save to Gold Layer
loan_summary_by_grade.write.format("delta").mode("overwrite").save("/FileStore/lending/gold/loan_summary_by_grade")



In [0]:
#Aggregation Example 2 – Default count by loan status
loan_status_counts = df_silver.groupBy("loan_status").count()

# Save to Gold Layer
loan_status_counts.write.format("delta").mode("overwrite").save("/FileStore/lending/gold/loan_status_counts")


In [0]:
# Aggregation Example 3 – Average DTI by home ownership
dti_by_home = df_silver.groupBy("home_ownership").agg(
    avg("dti").alias("avg_dti")
)

# Save to Gold Layer
dti_by_home.write.format("delta").mode("overwrite").save("/FileStore/lending/gold/dti_by_home_ownership")


In [0]:
#Show the first results
print("Loan Summary by Grade:")
loan_summary_by_grade.show()

print("Loan Status Counts:")
loan_status_counts.show()

print("Average DTI by Home Ownership:")
dti_by_home.show()

Loan Summary by Grade:
+-----+------------------+------------------+
|grade|   avg_loan_amount| avg_interest_rate|
+-----+------------------+------------------+
|    F| 19124.64653110048| 25.45409066985592|
|    E|17453.078391907933| 21.82965253356323|
|    B| 14173.33819852703| 10.67580623818657|
|    D| 15711.98300680591|18.143067189846647|
|    C|15038.083317821778|14.143688622323502|
|    A|14603.343209545825| 7.084545374764227|
|    G|20383.988740959896|28.074255424062848|
+-----+------------------+------------------+

Loan Status Counts:
+--------------------+-------+
|         loan_status|  count|
+--------------------+-------+
|          Fully Paid|1076751|
|             Default|     40|
|     In Grace Period|   8436|
|Does not meet the...|   1988|
|         Charged Off| 268558|
|  Late (31-120 days)|  21467|
|             Current| 878317|
|Does not meet the...|    761|
|   Late (16-30 days)|   4349|
|            Oct-2015|      1|
+--------------------+-------+

Average DTI by 